In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key=os.getenv("GROQ_API_KEY"), 
    model_name="llama-3.1-8b-instant"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission.


In [14]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://stripe.com/jobs/listing/software-engineering-new-grad/7176975/apply")
page_data = loader.load().pop().page_content

In [20]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# Define parser
parser = JsonOutputParser()

# Create prompt with format instructions
prompt_extract = PromptTemplate(
    template="""
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Extract the job postings and return them in JSON format.

    {format_instructions}
    """,
    input_variables=["page_data"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain_extract = prompt_extract | llm | parser
json_res = chain_extract.invoke({"page_data": page_data})

print(json_res)


{'Software Engineering, New Grad': {'title': 'Software Engineering, New Grad', 'description': 'No description provided', 'links': {'see_open_roles': 'See open roles'}}}
